## Olympics Prediction Model

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

In [3]:
!pip install tensorflow[and-cuda]

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 234.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 107.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 251.8 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 245.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 245.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 254.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 245.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 324.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 260.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 317.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 332.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 194.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 243.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 213.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 MB 210.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━

In [1]:
# Import packages

from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


/tmp/pip_packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Matplotlib created a temporary cache directory at /tmp/matplotlib-1qibefqn because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix

In [81]:
#Import Data

snowflake.get_connection(connection_name="ME_OLYMPICS_CNX")
data = get_dataframe("ATHLETE_EVENTS")
data

Exception occurred in getting snowflake connection: 'connectionSources'
Reading dataframe from snowflake native connector


,ID,NAME,SEX,AGE,HEIGHT,WEIGHT,TEAM,NOC,GAMES,YEAR,SEASON,CITY,SPORT,EVENT,MEDAL
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NA,NA,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NA,NA,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29,179,89,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NA
271112,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NA
271113,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NA
271114,135571,Tomasz Ireneusz ya,M,30,185,96,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NA


In [103]:
data_bkp = get_dataframe("ATHLETE_EVENTS")
data_bkp

Reading dataframe from snowflake native connector


,ID,NAME,SEX,AGE,HEIGHT,WEIGHT,TEAM,NOC,GAMES,YEAR,SEASON,CITY,SPORT,EVENT,MEDAL
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NA,NA,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NA,NA,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29,179,89,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NA
271112,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NA
271113,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NA
271114,135571,Tomasz Ireneusz ya,M,30,185,96,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NA


In [82]:
data['AGE'].value_counts().to_dict()

{'23': 21875,
 '24': 21720,
 '22': 20814,
 '25': 19707,
 '21': 19164,
 '26': 17675,
 '27': 16025,
 '20': 15258,
 '28': 14043,
 '19': 11643,
 '29': 11463,
 '30': 9488,
 'NA': 9474,
 '18': 8152,
 '31': 7559,
 '32': 6246,
 '17': 5376,
 '33': 4800,
 '34': 3985,
 '16': 3852,
 '35': 3133,
 '36': 2503,
 '15': 2203,
 '37': 1953,
 '38': 1612,
 '39': 1405,
 '40': 1210,
 '41': 953,
 '42': 866,
 '14': 837,
 '43': 793,
 '44': 683,
 '45': 584,
 '46': 429,
 '47': 408,
 '48': 407,
 '49': 362,
 '50': 278,
 '52': 244,
 '53': 200,
 '51': 199,
 '13': 187,
 '54': 162,
 '56': 131,
 '55': 104,
 '60': 88,
 '59': 87,
 '58': 84,
 '65': 84,
 '57': 69,
 '61': 68,
 '62': 62,
 '69': 60,
 '63': 56,
 '12': 39,
 '71': 33,
 '66': 31,
 '64': 30,
 '70': 28,
 '67': 25,
 '68': 25,
 '72': 24,
 '11': 13,
 '74': 12,
 '73': 8,
 '76': 7,
 '75': 4,
 '88': 3,
 '80': 3,
 '81': 2,
 '77': 2,
 '84': 1,
 '96': 1,
 '10': 1,
 '97': 1}

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      271116 non-null  int32 
 1   NAME    271116 non-null  object
 2   SEX     271116 non-null  object
 3   AGE     271116 non-null  object
 4   HEIGHT  271116 non-null  object
 5   WEIGHT  271116 non-null  object
 6   TEAM    271116 non-null  object
 7   NOC     271116 non-null  object
 8   GAMES   271116 non-null  object
 9   YEAR    271116 non-null  int16 
 10  SEASON  271116 non-null  object
 11  CITY    271116 non-null  object
 12  SPORT   271116 non-null  object
 13  EVENT   271116 non-null  object
 14  MEDAL   271116 non-null  object
dtypes: int16(1), int32(1), object(13)
memory usage: 28.4+ MB


In [84]:
data['AGE'][data['AGE']!='NA'].astype(int).median()

24.0

In [92]:
data['WEIGHT'][data['WEIGHT']!='NA'].astype(float).median()

70.0

In [89]:
data['HEIGHT'][data['HEIGHT']!='NA'].astype(int).median()

175.0

In [93]:
data['AGE'].replace('NA',int(data['AGE'][data['AGE']!='NA'].astype(int).median()))

0         24
1         23
2         24
3         34
4         21
          ..
271111    29
271112    27
271113    27
271114    30
271115    34
Name: AGE, Length: 271116, dtype: object

In [94]:
data['AGE'] = data['AGE'].replace('NA',str(int(data['AGE'][data['AGE']!='NA'].astype(int).median()))).astype(int)

In [95]:
data['HEIGHT'] = data['HEIGHT'].replace('NA',str(int(data['HEIGHT'][data['HEIGHT']!='NA'].astype(int).median()))).astype(int)

In [99]:
data['WEIGHT'] = data['WEIGHT'].replace('NA',str(float(data['WEIGHT'][data['WEIGHT']!='NA'].astype(float).median()))).astype(float)

In [100]:
data['WEIGHT'].value_counts().to_dict()

{70.0: 72500,
 60.0: 7994,
 75.0: 7810,
 68.0: 7284,
 65.0: 7236,
 72.0: 6252,
 80.0: 6214,
 73.0: 5937,
 63.0: 5869,
 64.0: 5764,
 62.0: 5611,
 67.0: 5263,
 58.0: 5243,
 78.0: 4995,
 74.0: 4809,
 66.0: 4764,
 57.0: 4640,
 82.0: 4186,
 76.0: 4109,
 55.0: 4106,
 77.0: 4035,
 56.0: 4029,
 69.0: 4027,
 59.0: 4012,
 85.0: 3909,
 61.0: 3817,
 71.0: 3637,
 54.0: 3507,
 52.0: 3450,
 90.0: 3371,
 79.0: 3146,
 84.0: 2945,
 83.0: 2925,
 50.0: 2587,
 81.0: 2557,
 86.0: 2536,
 53.0: 2386,
 88.0: 2142,
 48.0: 1965,
 87.0: 1876,
 95.0: 1842,
 51.0: 1839,
 100.0: 1544,
 92.0: 1473,
 89.0: 1438,
 93.0: 1409,
 91.0: 1324,
 49.0: 1110,
 47.0: 1039,
 94.0: 990,
 98.0: 892,
 96.0: 859,
 46.0: 777,
 45.0: 746,
 97.0: 684,
 105.0: 602,
 43.0: 521,
 102.0: 504,
 44.0: 442,
 110.0: 428,
 120.0: 395,
 40.0: 372,
 99.0: 370,
 42.0: 344,
 104.0: 314,
 115.0: 292,
 41.0: 259,
 103.0: 249,
 130.0: 203,
 125.0: 199,
 39.0: 196,
 38.0: 195,
 107.0: 187,
 108.0: 174,
 106.0: 174,
 37.0: 173,
 101.0: 166,
 36.0: 137,


In [101]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int32  
 1   NAME    271116 non-null  object 
 2   SEX     271116 non-null  object 
 3   AGE     271116 non-null  int64  
 4   HEIGHT  271116 non-null  int64  
 5   WEIGHT  271116 non-null  float64
 6   TEAM    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   GAMES   271116 non-null  object 
 9   YEAR    271116 non-null  int16  
 10  SEASON  271116 non-null  object 
 11  CITY    271116 non-null  object 
 12  SPORT   271116 non-null  object 
 13  EVENT   271116 non-null  object 
 14  MEDAL   271116 non-null  object 
dtypes: float64(1), int16(1), int32(1), int64(2), object(10)
memory usage: 28.4+ MB


In [6]:
tf.random.set_seed(100)

In [104]:
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      271116 non-null  int32  
 1   name    271116 non-null  object 
 2   sex     271116 non-null  object 
 3   age     271116 non-null  int64  
 4   height  271116 non-null  int64  
 5   weight  271116 non-null  float64
 6   team    271116 non-null  object 
 7   noc     271116 non-null  object 
 8   games   271116 non-null  object 
 9   year    271116 non-null  int16  
 10  season  271116 non-null  object 
 11  city    271116 non-null  object 
 12  sport   271116 non-null  object 
 13  event   271116 non-null  object 
 14  medal   271116 non-null  object 
dtypes: float64(1), int16(1), int32(1), int64(2), object(10)
memory usage: 28.4+ MB


# CREATING LABELS

In [105]:
data['medal'] = data['medal'].apply(lambda x: 1 if str(x) != 'nan' else 0)

In [106]:
data['age'].value_counts()

age
24    31194
23    21875
22    20814
25    19707
21    19164
      ...  
77        2
84        1
10        1
96        1
97        1
Name: count, Length: 74, dtype: int64

# Cleaning data

In [15]:
# converting datatypes
data = data.astype({'age': int, 'height': int, 'weight': int})
print(data.dtypes)


ValueError: invalid literal for int() with base 10: 'NA': Error while type casting for column 'height'

In [108]:
data = data.drop(['id', 'name', 'games'], axis=1)

In [109]:
data.isna().mean()

sex       0.0
age       0.0
height    0.0
weight    0.0
team      0.0
noc       0.0
year      0.0
season    0.0
city      0.0
sport     0.0
event     0.0
medal     0.0
dtype: float64

In [115]:
print("Total missing values:", data.isna().sum().sum())

Total missing values: 0


# Encoding Data

In [116]:
{column: len(data[column].unique()) for column in data.select_dtypes('object').columns}

{'sex': 2,
 'team': 1184,
 'noc': 230,
 'season': 2,
 'city': 42,
 'sport': 66,
 'event': 765}

In [117]:
def binary_encode(df, columns, positive_values):
    df = df.copy()
    for column, positive_value in zip(columns, positive_values):
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [118]:
data = binary_encode(
    data,
    columns=['sex', 'season'],
    positive_values=['M', 'Summer']
)

data = onehot_encode(
    data,
    columns=['team', 'noc', 'city', 'sport', 'event'],
    prefixes=['t', 'n', 'c', 's', 'e']
)

In [119]:
print("Remaining non-numeric columns:", len(data.select_dtypes('object').columns))

Remaining non-numeric columns: 0


In [ ]:
# Visualizing Correlations